In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import shutil
import array as arr
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D ,AveragePooling2D, Flatten, Dropout
from tensorflow.keras.layers import Dense,Reshape,InputLayer,UpSampling2D
from tensorflow.keras.optimizers import RMSprop,Adam,SGD
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import model_from_json
from tensorflow.keras.applications import MobileNetV2

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ronnybhau","key":"904903beb498d73a2d375f37491443f4"}'}

In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d andrewmvd/ocular-disease-recognition-odir5k

100% 1.25G/1.26G [00:15<00:00, 88.0MB/s]
100% 1.26G/1.26G [00:15<00:00, 88.9MB/s]


In [0]:
import os
import zipfile
local='/content/ocular-disease-recognition-odir5k.zip'
zipr=zipfile.ZipFile(local,'r')
zipr.extractall('/content')
zipr.close()

In [0]:
meta=pd.read_excel('/content/ODIR-5K/data.xlsx')

In [7]:
meta.head(25)

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1
3,3,66,Male,3_left.jpg,3_right.jpg,normal fundus,branch retinal artery occlusion,0,0,0,0,0,0,0,1
4,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1
5,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0
6,6,60,Male,6_left.jpg,6_right.jpg,macular epiretinal membrane,moderate non proliferative retinopathy，epireti...,0,1,0,0,0,0,0,1
7,7,60,Female,7_left.jpg,7_right.jpg,drusen,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1
8,8,59,Male,8_left.jpg,8_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
9,9,54,Male,9_left.jpg,9_right.jpg,normal fundus,vitreous degeneration,0,0,0,0,0,0,0,1


In [8]:
classes=["normal fundus",'retinopathy','glaucoma','cataract','age-related macular degeneration','hypertensive retinopathy','myopia']
for x in classes:
  print(x)

normal fundus
retinopathy
glaucoma
cataract
age-related macular degeneration
hypertensive retinopathy
myopia


NOTE:
age-related macular degeneration---stands for AMD


hypertensive retinopathy--- stands for hypertension

retinopathy ---stands for diabetes

In [9]:
to_create = [
    '/content/eye',
     '/content/eye/age-related macular degeneration',
     '/content/eye/cataract',
     '/content/eye/retinopathy',
     '/content/eye/glaucoma',
     '/content/eye/hypertensive retinopathy',
     '/content/eye/myopia',
      '/content/eye/normal fundus',
       '/content/eye/other',

]

for directory in to_create:
    try:
        os.mkdir(directory)
        print(directory, 'created')
    except:
        print(directory, 'failed')

/content/eye created
/content/eye/age-related macular degeneration created
/content/eye/cataract created
/content/eye/retinopathy created
/content/eye/glaucoma created
/content/eye/hypertensive retinopathy created
/content/eye/myopia created
/content/eye/normal fundus created
/content/eye/other created


In [0]:
a = arr.array('i', [0, 0, 0,0,0,0,0,0]) 

In [0]:

for x,k in zip(meta['Left-Diagnostic Keywords'],meta['ID']):
 comma=x.count(chr(65292))
 path="/content/ODIR-5K/Training Images/"+ str(k) +"_left.jpg"
 m=0
 i=0  
 for j in classes: 
  if (x.count(j)>0):
   if(j=='retinopathy'):
     if(x.find("hypertensive retinopathy") != -1):
       if (x.count(j)>1):
         shutil.copy(path,'/content/eye/'+j)
         a[i]+=1
         m+=1
     else:
       shutil.copy(path,'/content/eye/'+j)
       a[i]+=1
       m+=1
   else:
     if(j=="normal fundus"):
       m+=1
       if(comma==0):
          shutil.copy(path,'/content/eye/'+j)
          a[i]+=1
       elif(comma==1):
         if(x.find("lens dust") != -1):
           shutil.copy(path,'/content/eye/'+j)
           a[i]+=1
     
     else:
      shutil.copy(path,'/content/eye/'+j)
      a[i]+=1
      m+=1
  i+=1 
 if(m<comma+1):
   if(x.find("lens dust") != -1):
     if(m<comma):
       shutil.copy(path,'/content/eye/other')
       a[7]+=1
   else:
    shutil.copy(path,'/content/eye/other')
    a[7]+=1
    

In [0]:
for x,k in zip(meta['Right-Diagnostic Keywords'],meta['ID']):
 comma=x.count(chr(65292))
 path="/content/ODIR-5K/Training Images/"+ str(k) +"_right.jpg"
 m=0
 i=0  
 for j in classes:
  if (x.count(j)>0):
   if(j=='retinopathy'):
     if(x.find("hypertensive retinopathy") != -1):
       if (x.count(j)>1):
         shutil.copy(path,'/content/eye/'+j)
         a[i]+=1
         m+=1
     else:
       shutil.copy(path,'/content/eye/'+j)
       a[i]+=1
       m+=1
   else:
     if(j=="normal fundus"):
       m+=1
       if(comma==0):
          shutil.copy(path,'/content/eye/'+j)
          a[i]+=1
       elif(comma==1):
         if(x.find("lens dust") != -1):
           shutil.copy(path,'/content/eye/'+j)
           a[i]+=1
     
     else:
      shutil.copy(path,'/content/eye/'+j)
      a[i]+=1
      m+=1
  i+=1 
 if(m<comma+1):
   if(x.find("lens dust") != -1):
     if(m<comma):
       shutil.copy(path,'/content/eye/other')
       a[7]+=1
   else:
    shutil.copy(path,'/content/eye/other')
    a[7]+=1

In [0]:
#shutil.rmtree('/content/eye', ignore_errors=True)

In [15]:
path = '{}/eye'.format(os.getcwd())
path

'/content/eye'

In [16]:
datagen = ImageDataGenerator(rescale=1.0/255)
train_gen = datagen.flow_from_directory(
    path,
    target_size=(224,224),
    batch_size=64,
    class_mode='categorical'
)

Found 7524 images belonging to 8 classes.


In [23]:
model = Sequential()
model.add(InputLayer((224,224,3)))
model.add(Conv2D(filters=16,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
#model.add(Dropout(0.5))
model.add(Conv2D(filters=32,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
#model.add(Dropout(0.5))
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
#model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(8,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 32)      4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 26, 26, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 43264)             0

In [25]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')
history = model.fit(train_gen,epochs=2
                    0)

Epoch 1/50
118/118 [==============================] - 374s 3s/step - loss: 1.6069 - accuracy: 0.4094
Epoch 2/50
118/118 [==============================] - 376s 3s/step - loss: 1.5478 - accuracy: 0.4156
Epoch 3/50
118/118 [==============================] - 384s 3s/step - loss: 1.4916 - accuracy: 0.4285
Epoch 4/50
118/118 [==============================] - 359s 3s/step - loss: 1.4180 - accuracy: 0.4428
Epoch 5/50
118/118 [==============================] - 357s 3s/step - loss: 1.3733 - accuracy: 0.4551
Epoch 6/50
118/118 [==============================] - 357s 3s/step - loss: 1.3224 - accuracy: 0.4688
Epoch 7/50
118/118 [==============================] - 357s 3s/step - loss: 1.2607 - accuracy: 0.4947
Epoch 8/50
118/118 [==============================] - 364s 3s/step - loss: 1.1982 - accuracy: 0.5183
Epoch 9/50
118/118 [==============================] - 363s 3s/step - loss: 1.1392 - accuracy: 0.5393
Epoch 10/50
118/118 [==============================] - 355s 3s/step - loss: 1.0528 - accura

In [0]:
model_json = model.to_json()
with open('ocular.json','w') as json_file:
  json_file.write(model_json)
model.save_weights('ocular_model.h5')